In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [3]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineUi.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
5,131,25296961,커피향,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,142070,1111.0,NaN,1,NaN,127.017727,37.633457,우이신설 화계,86.776146,NaN
25,1329,28508122,그라찌에,국민대학교과학관,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,136702,2707.0,NaN,NaN,NaN,126.998518,37.610184,우이신설 정릉,355.714873,NaN
26,1384,15480069,달콤퐁당,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,136071,2850.0,NaN,1,NaN,127.022253,37.588876,우이신설 보문,319.837361,NaN
38,2211,19991089,언덕위에카파,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,136100,2708.0,NaN,1,NaN,127.004063,37.611412,우이신설 정릉,279.645897,NaN
42,2384,16129339,간단하지만특별한화피디Cafe,인피니티,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,136702,2707.0,NaN,NaN,NaN,126.998518,37.610184,우이신설 정릉,355.714873,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2319,132745,16990548,앨수애뇨부리또하우스,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,132885,1368.0,NaN,1,NaN,127.013581,37.650560,우이신설 솔밭공원,380.695138,NaN
2334,133890,23212037,커피에반하다,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,136110,2724.0,NaN,1,119,127.019657,37.610713,우이신설 삼양사거리,758.974677,NaN
2336,133971,17106038,19티정릉,솔샘점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,136849,2711.0,NaN,NaN,NaN,127.007893,37.612940,우이신설 북한산보국문,133.506276,NaN
2337,133972,17103908,나,스콘카페안,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,136849,2713.0,NaN,NaN,NaN,127.013556,37.612131,우이신설 솔샘,403.715684,NaN


In [4]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
5,커피향,커피점/카페,커피전문점/카페/다방,서울특별시 강북구 삼양로 336,우이신설 화계,86.776146,NaN,화계,화계역 커피향
25,그라찌에,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 정릉로 77,우이신설 정릉,355.714873,NaN,정릉,정릉역 그라찌에
26,달콤퐁당,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 인촌로13길 57,우이신설 보문,319.837361,NaN,보문,보문역 달콤퐁당
38,언덕위에카파,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 솔샘로15다길 17,우이신설 정릉,279.645897,NaN,정릉,정릉역 언덕위에카파
42,간단하지만특별한화피디Cafe,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 정릉로 77,우이신설 정릉,355.714873,NaN,정릉,정릉역 간단하지만특별한화피디Cafe
...,...,...,...,...,...,...,...,...,...
2319,앨수애뇨부리또하우스,커피점/카페,커피전문점/카페/다방,서울특별시 도봉구 삼양로142길 3,우이신설 솔밭공원,380.695138,NaN,솔밭공원,솔밭공원역 앨수애뇨부리또하우스
2334,커피에반하다,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 삼양로13길 45,우이신설 삼양사거리,758.974677,NaN,삼양사거리,삼양사거리역 커피에반하다
2336,19티정릉,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 솔샘로 74,우이신설 북한산보국문,133.506276,NaN,북한산보국문,북한산보국문역 19티정릉
2337,나,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 서경로 106-2,우이신설 솔샘,403.715684,NaN,솔샘,솔샘역 나


In [5]:
items = data["검색어"]
items

5                   화계역 커피향
25                 정릉역 그라찌에
26                 보문역 달콤퐁당
38               정릉역 언덕위에카파
42      정릉역 간단하지만특별한화피디Cafe
               ...         
2319       솔밭공원역 앨수애뇨부리또하우스
2334          삼양사거리역 커피에반하다
2336          북한산보국문역 19티정릉
2337                  솔샘역 나
2346           가오리역 커피와단풍나무
Name: 검색어, Length: 328, dtype: object

In [6]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [7]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/328
식당 존재 x
진행상황 : 2/328
식당 존재 x
진행상황 : 3/328
식당 존재
https://place.map.kakao.com/2009759375
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/328
식당 존재 x
진행상황 : 5/328
식당 존재 x
진행상황 : 6/328
식당 존재 x
진행상황 : 7/328
식당 존재 x
진행상황 : 8/328
식당 존재 x
진행상황 : 9/328
식당 존재 x
진행상황 : 10/328
식당 존재
https://place.map.kakao.com/1933920268
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 11/328
식당 존재 x
진행상황 : 12/328
식당 존재 x
진행상황 : 13/328
식당 존재 x
진행상황 : 14/328
식당 존재 x
진행상황 : 15/328
식당 존재 x
진행상황 : 16/328
식당 존재 x
진행상황 : 17/328
식당 존재 x
진행상황 : 18/328
식당 존재
https://place.map.kakao.com/1247378417
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 19/328
식당 존재 x
진행상황 : 20/328
식당 존재 x
진행상황 : 21/328
식당 존재
https://place.map.kakao.com/840477594
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 22/328
식당 존재 x
진행상황 : 23/328
식당 존재 x
진행상황 : 24/328
식당 존재
https://place.map.kakao.com/22650974
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 25/328
식당 존재 x
진행상황 : 26/328
식당 존재
https://place.map.kakao.com/27345890
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 27/328
식당 존재 x
진행상

식당 존재
https://place.map.kakao.com/799911250
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 213/328
식당 존재 x
진행상황 : 214/328
식당 존재
https://place.map.kakao.com/1559420005
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 215/328
식당 존재 x
진행상황 : 216/328
식당 존재 x
진행상황 : 217/328
식당 존재 x
진행상황 : 218/328
식당 존재 x
진행상황 : 219/328
식당 존재 x
진행상황 : 220/328
식당 존재
https://place.map.kakao.com/647077430
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 221/328
식당 존재 x
진행상황 : 222/328
식당 존재 x
진행상황 : 223/328
식당 존재 x
진행상황 : 224/328
식당 존재 x
진행상황 : 225/328
식당 존재
https://place.map.kakao.com/21411227
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 226/328
식당 존재
https://place.map.kakao.com/1180930469
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 227/328
식당 존재 x
진행상황 : 228/328
식당 존재 x
진행상황 : 229/328
식당 존재 x
진행상황 : 230/328
식당 존재 x
진행상황 : 231/328
식당 존재 x
진행상황 : 232/328
식당 존재
https://place.map.kakao.com/15608680
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 233/328
식당 존재 x
진행상황 : 234/328
식당 존재 x
진행상황 : 235/328
식당 존재 x
진행상황 : 236/328
식당 존재 x
진행상황 : 237/328
식당 존재 x
진행상황 : 238/328
식당 존재 x
진행상황 : 239

In [8]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,mudt04,보문역 달콤퐁당,1점,2021.02.19.
1,15frc1r,보문역 달콤퐁당,3점,2021.01.29.
2,14ic84m,보문역 달콤퐁당,5점,2020.08.01.
3,fsu7gu,보문역 달콤퐁당,5점,2020.05.17.
4,v2chb1,보문역 달콤퐁당,5점,2020.04.29.
...,...,...,...,...
341,mvh6pp,보문역 캘린더커피스탠드,2점,2020.07.26.
342,18m4p64,보문역 캘린더커피스탠드,5점,2020.05.24.
343,ndn4ku,보문역 캘린더커피스탠드,5점,2020.03.14.
344,m4i3im,보문역 캘린더커피스탠드,1점,2020.03.05.


In [9]:
img_elem

,ItemID,img_url
0,보문역 달콤퐁당,None
1,보문역 달콤퐁당,None
2,보문역 달콤퐁당,None
3,보문역 달콤퐁당,None
4,보문역 달콤퐁당,None
...,...,...
341,보문역 캘린더커피스탠드,None
342,보문역 캘린더커피스탠드,None
343,보문역 캘린더커피스탠드,None
344,보문역 캘린더커피스탠드,None


In [10]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
5,커피향,커피점/카페,커피전문점/카페/다방,서울특별시 강북구 삼양로 336,우이신설 화계,86.776146,NaN,화계,화계역 커피향
25,그라찌에,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 정릉로 77,우이신설 정릉,355.714873,NaN,정릉,정릉역 그라찌에
26,달콤퐁당,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 인촌로13길 57,우이신설 보문,319.837361,https://place.map.kakao.com/2009759375,보문,보문역 달콤퐁당
38,언덕위에카파,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 솔샘로15다길 17,우이신설 정릉,279.645897,None,정릉,정릉역 언덕위에카파
42,간단하지만특별한화피디Cafe,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 정릉로 77,우이신설 정릉,355.714873,None,정릉,정릉역 간단하지만특별한화피디Cafe
...,...,...,...,...,...,...,...,...,...
2319,앨수애뇨부리또하우스,커피점/카페,커피전문점/카페/다방,서울특별시 도봉구 삼양로142길 3,우이신설 솔밭공원,380.695138,None,솔밭공원,솔밭공원역 앨수애뇨부리또하우스
2334,커피에반하다,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 삼양로13길 45,우이신설 삼양사거리,758.974677,None,삼양사거리,삼양사거리역 커피에반하다
2336,19티정릉,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 솔샘로 74,우이신설 북한산보국문,133.506276,None,북한산보국문,북한산보국문역 19티정릉
2337,나,커피점/카페,커피전문점/카페/다방,서울특별시 성북구 서경로 106-2,우이신설 솔샘,403.715684,https://place.map.kakao.com/320375633,솔샘,솔샘역 나


In [11]:
img_elem

,ItemID,img_url
0,보문역 달콤퐁당,None
1,보문역 달콤퐁당,None
2,보문역 달콤퐁당,None
3,보문역 달콤퐁당,None
4,보문역 달콤퐁당,None
...,...,...
341,보문역 캘린더커피스탠드,None
342,보문역 캘린더커피스탠드,None
343,보문역 캘린더커피스탠드,None
344,보문역 캘린더커피스탠드,None


In [12]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,mudt04,보문역 달콤퐁당,1점,2021.02.19.
1,15frc1r,보문역 달콤퐁당,3점,2021.01.29.
2,14ic84m,보문역 달콤퐁당,5점,2020.08.01.
3,fsu7gu,보문역 달콤퐁당,5점,2020.05.17.
4,v2chb1,보문역 달콤퐁당,5점,2020.04.29.
...,...,...,...,...
341,mvh6pp,보문역 캘린더커피스탠드,2점,2020.07.26.
342,18m4p64,보문역 캘린더커피스탠드,5점,2020.05.24.
343,ndn4ku,보문역 캘린더커피스탠드,5점,2020.03.14.
344,m4i3im,보문역 캘린더커피스탠드,1점,2020.03.05.


In [13]:
review_elem

,ItemID,review
0,보문역 달콤퐁당,별로예요
1,보문역 달콤퐁당,꼬끄가 너무 파사삭이에요
2,보문역 달콤퐁당,성신여대 근처 마카롱 가게중 젤 맛있어요!
3,보문역 달콤퐁당,마카롱 맛있어요
4,보문역 달콤퐁당,케이크 너무예뻐서 5번째 재주문중..ㅠㅠㅠ
...,...,...
341,보문역 캘린더커피스탠드,"커피 맛, 응대 모두 좋았는데.... 시끄러운 손님에 대한 다른 손님들이 스트레스 ..."
342,보문역 캘린더커피스탠드,분위기도 좋고 커피가 정말 맛있어요 안암 근처에서 제일 맛있는 카페인듯
343,보문역 캘린더커피스탠드,커피 좋습니다
344,보문역 캘린더커피스탠드,멀리서 일부러 찾아갔는데 인스타그램 공지 한 줄도 없이 휴무였음


In [14]:
img_elem.head()

,ItemID,img_url
0,보문역 달콤퐁당,None
1,보문역 달콤퐁당,None
2,보문역 달콤퐁당,None
3,보문역 달콤퐁당,None
4,보문역 달콤퐁당,None


In [15]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




      UserID        ItemID Rating    Timestamp
331  1ba3aqb  보문역 캘린더커피스탠드     3점  2021.04.15.
332   cmnqa6  보문역 캘린더커피스탠드     5점  2021.03.05.
333  14qd4fr  보문역 캘린더커피스탠드     5점  2021.02.02.
334  1b204ub  보문역 캘린더커피스탠드     5점  2021.01.27.
335   vas8tb  보문역 캘린더커피스탠드     5점  2021.01.22.
336   mugono  보문역 캘린더커피스탠드     5점  2021.01.17.
337   bh7ism  보문역 캘린더커피스탠드     5점  2020.12.05.
338   bfad28  보문역 캘린더커피스탠드     5점  2020.09.26.
339  10cvpe2  보문역 캘린더커피스탠드     5점  2020.09.08.
340  11v70jn  보문역 캘린더커피스탠드     5점  2020.08.02.
341   mvh6pp  보문역 캘린더커피스탠드     2점  2020.07.26.
342  18m4p64  보문역 캘린더커피스탠드     5점  2020.05.24.
343   ndn4ku  보문역 캘린더커피스탠드     5점  2020.03.14.
344   m4i3im  보문역 캘린더커피스탠드     1점  2020.03.05.
345   ljr0r8  보문역 캘린더커피스탠드     5점  2019.09.18.
           ItemID img_url
331  보문역 캘린더커피스탠드    None
332  보문역 캘린더커피스탠드    None
333  보문역 캘린더커피스탠드    None
334  보문역 캘린더커피스탠드    None
335  보문역 캘린더커피스탠드    None
336  보문역 캘린더커피스탠드      []
337  보문역 캘린더커피스탠드    None
338  보문역 캘린더커피스탠드    None
339  보문역 캘린더커피

In [16]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineUi_cafe(final).csv')

In [17]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineUi_cafe_User(final).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineUi_cafe_img(final).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/llineUi_cafe_review(final).csv')

In [18]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


47

In [19]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lineUi_url.csv')